* 토픽 모델링의 데표적인 알고리즘
* 문서들은 토픽들의 혼합으로 구성되어져 있으며, 토픽들은 확률 분포에 기반하여 단어들을 생성한다고 가정
* 문서가 생성되던 과정을 역추적과정을 역추적
* 각 문서의 토픽 분포와 각 토픽 내의 단어 분포를 추정
* 문서의 집합으로부터 어떤 토픽이 존재하는지를 알아내기 위한 알고리즘
* 단어의 순서는 신경쓰지 않는다.

### LDA의 수행 과정
1. 사용자는 알고리즘에게 토픽의 개수 k를 알려준다.
    * 토픽의 개수 k를 입력받으면, k개의 토픽이 M개의 전체 문서에 걸쳐 분포되어 있다고 가정
2. 모든 단어를 k개 중 하나의 토픽에 할당한다.
    * 랜덤으로 할당해 결과는 전부 틀린 상태다.
3. 이제 모든 문서의 모든 단어데 대해 아래의 사항을 반복 진행한다.

    1) 어떤 문서의 각 단어 w는 자신은 잘못된 토픽에 할당되어져 있지만, 다른 단어들은 전부 올바른 토픽에 할당되어져 있다고 가정, 그리고 단어 w는 아래의 두 가지 기준에 따라 토픽 재할당
    
        * p(topic t | document d) : 문서 d의 단어들 중 토픽 t에 해당하는 단어들의 비율
        * p(word w | topic t) : 각 토픽들 t에서 해당 단어 w의 분포
4. 이를 반복하면 모든 할당이 완료된 수렴 상태가 된다.

### 잠재 디리클레 할당과 잠재 의미 분석의 차이
**LSA** : DTM을 차원 축소 하여 축소 차원에서 근접 단어들을 토픽으로 묶는다.


**LDA** : 단어가 특정 토픽에 존재할 확률과 문서에 특정 토픽이 존재할 확률을 결합확률로 추정하여 토픽을 추출한다.

## [실습]
*사이킷런이 아닌 gensim을 사용*

### 1) 정수 인코딩과 단어 집합 만들기

In [1]:
## 이전 챕터와 중복 내용

# 뉴스 그룹 데이터

import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers','footers','quotes'))
documents = dataset.data

C:\Users\A\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\A\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\A\anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# 전처리

news_df = pd.DataFrame({'document':documents})

# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]"," ")

# 길이가 3이하인 단어 제거
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

# 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

C:\Users\A\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [3]:
# 토큰화 -> 불용어 제거

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [4]:
tokenized_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

In [5]:
# (각 단어를 정수 인코딩한 값, 단어의 빈도 수)의 형태로 바꾸기

from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [6]:
print(dictionary[66])

faith


In [7]:
len(dictionary)

64281

### 2) LDA 모델 훈련시키기

In [8]:
# 토픽의 개수 == 20

import gensim
NUM_TOPICS = 20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15) # passes는 알고리즘의 동작 횟수
topics = ldamodel.print_topics(num_words=4) # 4개의 단어만 풀력
for topic in topics:
    print(topic)

(0, '0.023*"henrik" + 0.013*"karabakh" + 0.013*"plane" + 0.011*"azeris"')
(1, '0.008*"jays" + 0.007*"spots" + 0.007*"copper" + 0.006*"strip"')
(2, '0.008*"used" + 0.008*"power" + 0.006*"engine" + 0.005*"ground"')
(3, '0.011*"windows" + 0.008*"drive" + 0.008*"system" + 0.007*"using"')
(4, '0.022*"armenian" + 0.018*"turkish" + 0.016*"jews" + 0.016*"armenians"')
(5, '0.043*"jesus" + 0.021*"bible" + 0.021*"church" + 0.018*"christ"')
(6, '0.016*"entry" + 0.014*"output" + 0.013*"encryption" + 0.011*"chip"')
(7, '0.045*"space" + 0.017*"nasa" + 0.010*"launch" + 0.009*"earth"')
(8, '0.016*"available" + 0.011*"mail" + 0.011*"version" + 0.011*"software"')
(9, '0.022*"price" + 0.020*"sale" + 0.018*"scsi" + 0.017*"shipping"')
(10, '0.021*"would" + 0.018*"like" + 0.016*"know" + 0.015*"think"')
(11, '0.011*"people" + 0.010*"would" + 0.006*"many" + 0.005*"even"')
(12, '0.017*"gordon" + 0.016*"picture" + 0.016*"pitt" + 0.015*"banks"')
(13, '0.014*"would" + 0.009*"fire" + 0.006*"koresh" + 0.005*"even"')

### 3) LDA 시각화 하기

In [16]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

C:\Users\A\anaconda3\lib\site-packages\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\A\anaconda3\lib\site-packages\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))


### 4) 문서 별 토픽 분포 보기

In [17]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(2, 0.020036552), (4, 0.23997968), (5, 0.01955572), (11, 0.528932), (13, 0.100214675), (14, 0.07977185)]
1 번째 문서의 topic 비율은 [(1, 0.05394788), (5, 0.06815487), (10, 0.58005893), (11, 0.24854448), (18, 0.0295326)]
2 번째 문서의 topic 비율은 [(2, 0.08705511), (10, 0.31519437), (11, 0.58333117)]
3 번째 문서의 topic 비율은 [(0, 0.019171555), (2, 0.07016238), (3, 0.091782495), (6, 0.27891064), (10, 0.114145055), (11, 0.38857985), (13, 0.027067047)]
4 번째 문서의 topic 비율은 [(6, 0.31118277), (10, 0.37096506), (14, 0.28636128)]


In [18]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [19]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,11.0,0.5289,"[(2, 0.020036219), (4, 0.23997241), (5, 0.0195..."
1,1,10.0,0.5801,"[(1, 0.05394788), (5, 0.068159305), (10, 0.580..."
2,2,11.0,0.5833,"[(2, 0.08705474), (10, 0.31520483), (11, 0.583..."
3,3,11.0,0.3885,"[(0, 0.019174825), (2, 0.07021944), (3, 0.0916..."
4,4,10.0,0.3710,"[(6, 0.31118628), (10, 0.37095755), (14, 0.286..."
5,5,5.0,0.4278,"[(5, 0.42780235), (10, 0.3084772), (11, 0.1533..."
6,6,3.0,0.4083,"[(2, 0.28165826), (3, 0.4082692), (10, 0.23075..."
7,7,11.0,0.5943,"[(5, 0.022124138), (9, 0.02392045), (10, 0.266..."
8,8,10.0,0.2341,"[(3, 0.12373228), (5, 0.19168808), (6, 0.09682..."
9,9,10.0,0.6080,"[(2, 0.21395238), (4, 0.017656883), (6, 0.0553..."
